# Get Song Recommendations 💃

In [ ]:
def save(model, filename = "filename.pickle"): 
    with open(filename, "wb") as f:
        pickle.dump(model, f)

def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
    except FileNotFoundError: 
        print("File not found!")

In [ ]:
#kmeans = KMeans(n_clusters=84, random_state=1234, n_init=150);
#kmeans.fit(df_scaled.values);

#save(kmeans, "kmeans.pickle")

In [ ]:
from sklearn.preprocessing import StandardScaler
import random
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import string
import Levenshtein as lev
from IPython.display import IFrame
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config
import numpy as np
import pickle
from sklearn.cluster import KMeans


def scrape(x):
    url = x
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    songs = []
    artist = []
    for i in soup.select("li.o-chart-results-list__item > h3"):
        songs.append(i.get_text(strip=True).lower())
    for i in soup.select("li > span.c-label.a-no-trucate"):
        artist.append(i.get_text(strip=True))
    
    df = pd.DataFrame({'Song': songs, 'Artist': artist})
    return df


df = scrape("https://www.billboard.com/charts/hot-100/")
df_cluster = pd.read_csv("songs.csv")
df_cluster = df_cluster.drop(columns="cluster",axis=1)
df_train = df_cluster.drop(columns=["mode","id"],axis=1)

scaler = StandardScaler()
scaler.fit(df_train)
df_scaled = scaler.transform(df_train)
df_scaled = pd.DataFrame(df_scaled, columns = df_train.columns)


kmeans = load("kmeans.pickle")

labels = kmeans.labels_
df_cluster["cluster"] = labels

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

def play_song(song):
    
    results = sp.search(q=song,limit=1,market="GB")
    track_id = results['tracks']["items"][0]["id"]
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
       )


user = input("Enter your song:")
user = user.lower()

for char in string.punctuation:
    user = user.replace(char, "")

if user in list(df["Song"]):
    df_new = df[df["Song"]!=user]
    x = df_new["Song"][random.randint(0, len(df_new))]
    print("Your song is hot!")
    display(play_song(user))
    print("Here is another new hot song:",x.upper())
    display(play_song(x))
    
else:
    similar = True
    df_new = pd.DataFrame()
    for song in list(df['Song']):
        if lev.distance(user, song) <= 2:
            display(play_song(song))
            answer = input(f' Did you mean {song}? Yes or No:')
            
            if (answer.lower()!="yes") & (answer.lower()!="no"):
                print("Invalid entry.")
                answer = input("Please either enter YES or NO:")
            
            if answer.lower() == 'yes':
                df_new = df[df["Song"]!=song]
                x = df_new["Song"][random.randint(0, len(df_new))]
                print("Your song is hot!")
                print("Here is another new hot song:",x.upper())
                display(play_song(x))
                similar = False
            else:
                print("Sorry, your song is not hot.")
                answer_1 = input("Would you like recommendations based on your song? Yes or No:")
                
                if (answer_1.lower()!="yes") & (answer_1.lower()!="no"):
                    print("Invalid entry.")
                    answer_1 = input("Please either enter YES or NO:")
                
                if answer_1.lower() == 'yes':
                    results = sp.search(q=user,limit=3,market="GB")
                    for i in results['tracks']["items"]:
                        track_id = i["id"]
                        display(IFrame(src="https://open.spotify.com/embed/track/"+track_id,
                        width="320",height="80",
                        frameborder="0",
                        allowtransparency="true",
                        allow="encrypted-media",
                        )) 
                        an = input("Is this the song you entered? Yes or No:")
                        
                        if (an.lower()!="yes") & (an.lower()!="no"):
                            print("Invalid entry.")
                            an = input("Please either enter YES or NO:")
                    
                        if an.lower() == 'yes':

                            x = pd.DataFrame(sp.audio_features(track_id)[0], index=[0])
                            x = x.drop(columns=["mode","type","id","uri","track_href","analysis_url",
                                            "duration_ms","time_signature"],axis=1)
                            x_scaled = scaler.transform(x)
                            df_1 = df_cluster[(df_cluster["cluster"]==kmeans.predict(x_scaled)[0]) 
                                      & (df_cluster["id"]!=track_id)].reset_index()
                            recom = df_1["id"][random.randint(0, len(df_1))]
                            print("Recommendation:")
                            display(IFrame(src="https://open.spotify.com/embed/track/"+recom,width="320",height="80",
                            frameborder="0",
                            allowtransparency="true",
                            allow="encrypted-media",
                            )) 
                            answer_2 = input("Would you like more? Yes or No:")
                            
                            if (answer_2.lower()!="yes") & (answer_2.lower()!="no"):
                                print("Invalid entry.")
                                answer_2 = input("Please either enter YES or NO:")
                            
                            if answer_2.lower() == 'yes':
                                df_1 = df_1[df_1["id"]!=recom]
                                recom_1 = df_1["id"][random.randint(0, len(df_1))]
                                display(IFrame(src="https://open.spotify.com/embed/track/"+recom_1,width="320",height="80",
                                frameborder="0",
                                allowtransparency="true",
                                allow="encrypted-media",
                                )) 
                                similar = False
                                break
                            else:
                                print("Have a nice day!")
                                similar = False
                                break
                        
                        else:
                            continue
                    
                else:
                    print("Alrighty!")
                    similar = False
                    
            
    if similar:
        n = True
        results = sp.search(q=user,limit=3,market="GB")
        for i in results['tracks']["items"]:
            track_id = i["id"]
            display(IFrame(src="https://open.spotify.com/embed/track/"+track_id,
            width="320",height="80",
            frameborder="0",
            allowtransparency="true",
            allow="encrypted-media",
            )) 
            an = input("Is this the song you entered? Yes or No:")
            
            if (an.lower()!="yes") & (an.lower()!="no"):
                print("Invalid entry.")
                an = input("Please either enter YES or NO:")
            
            if an.lower() == 'yes':
                x = pd.DataFrame(sp.audio_features(track_id)[0], index=[0])
                x = x.drop(columns=["mode","type","id","uri","track_href","analysis_url",
                                    "duration_ms","time_signature"],axis=1)
                x_scaled = scaler.transform(x)
                df_1 = df_cluster[(df_cluster["cluster"]==kmeans.predict(x_scaled)[0]) 
                                      & (df_cluster["id"]!=track_id)].reset_index()
                recom = df_1["id"][random.randint(0, len(df_1))]
                print("Recommendation:")
                display(IFrame(src="https://open.spotify.com/embed/track/"+recom,width="320",height="80",
                frameborder="0",
                allowtransparency="true",
                allow="encrypted-media",
                ))
                answer_2 = input("Would you like more? Yes or No:")
                
                if (answer_2.lower()!="yes") & (answer_2.lower()!="no"):
                    print("Invalid entry.")
                    answer_2 = input("Please either enter YES or NO:")
                
                
                if answer_2.lower() == 'yes':
                    df_1 = df_1[df_1["id"]!=recom]
                    recom_1 = df_1["id"][random.randint(0, len(df_1))]
                    display(IFrame(src="https://open.spotify.com/embed/track/"+recom_1,width="320",height="80",
                    frameborder="0",
                    allowtransparency="true",
                    allow="encrypted-media",
                    )) 
                    n = False
                    break
                else:
                    print("Have a nice day!")
                    n = False
                    break        
            else:
                continue
                    
        if n:
            print("Sorry, we do not have any recommendations for you.")
                